<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/autoencoder2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AutoEncoder on fashion MNIST




In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models, datasets, callbacks
import tensorflow.keras.backend as K



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#0. Parameters

In [3]:
IMAGE_SIZE = 32
CHANNELS = 1
BATCH_SIZE = 100
BUFFER_SIZE = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 2
EPOCHS = 3



#1. Prepare the data

In [4]:
#Load the data
(x_train , y_train) , (x_test , y_test) = datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
#Preprocess the data

def preprocess(imgs):
  """
  Normalize and reshape the images
  """
  imgs = imgs.astype("float32") / 255.0
  imgs = np.pad(imgs, ((0,0) ,(2,2) , (2,2)) , constant_values = 0.0)
  imgs = np.expand_dims(imgs, -1)
  return imgs

x_train = preprocess(x_train)
x_test = preprocess(x_test)



#2. Build the autoencoder

In [7]:
#Encoder
encoder_input = layers.Input(
    shape = (IMAGE_SIZE , IMAGE_SIZE , CHANNELS) , name = "encoder_input"
)
x = layers.Conv2D(32, (3,3), strides = 2, activation = "relu" , padding = "same")(
    encoder_input
)
x = layers.Conv2D(64, (3,3) , strides = 2, activation = "relu" , padding = "same")(x)
x = layers.Conv2D(128, (3,3) , strides = 2, activation = "relu" , padding = "same")(x)
shape_before_flattening = K.int_shape(x)[1:]  #the decoder will need this

x = layers.Flatten()(x)
encoder_output = layers.Dense(EMBEDDING_DIM , name= "encoder_output")(x)

encoder = models.Model(encoder_input, encoder_output)
encoder.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 conv2d_4 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 encoder_output (Dense)      (None, 2)                 4098      
                                                                 
Total params: 96770 (378.01 KB)
Trainable params: 96770 (378.

In [8]:
#Decoder
decoder_input = layers.Input(shape=(EMBEDDING_DIM, ), name = "decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(
    128, (3,3), strides = 2, activation = "relu" , padding = "same"
)(x)
x = layers.Conv2DTranspose(
    64, (3,3) , strides = 2 , activation = "relu" , padding = "same"
)(x)
x = layers.Conv2DTranspose(
    32, (3,3) , strides = 2, activation = "relu" , padding = "same"
)(x)
decoder_output = layers.Conv2D(
    CHANNELS,
    (3,3),
    strides = 1,
    activation = "sigmoid",
    padding = "same",
    name = "decoder_output",
)(x)

decoder = models.Model(decoder_input , decoder_output)
decoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 2048)              6144      
                                                                 
 reshape (Reshape)           (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 8, 8, 128)         147584    
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 16, 16, 64)        73792     
 Transpose)                                                      
                                                                 
 conv2d_transpose_2 (Conv2D  (None, 32, 32, 32)        1846

In [9]:
#Autoencoder
autoencoder = models.Model(
    encoder_input, decoder(encoder_output)
)
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 conv2d_4 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 encoder_output (Dense)      (None, 2)                 4098      
                                                                 
 model_1 (Functional)        (None, 32, 32, 1)         2462

#3. Train the autoencoder

In [10]:
#compile the autoencoder
autoencoder.compile(optimizer = "adam" , loss = "binary_crossentropy")


In [11]:
#Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath = "./checkpoint",
    save_weights_only = False,
    save_freq = "epoch",
    monitor = "less",
    mode = "min" ,
    save_best_only = True,
    verbose = 0,
)
tensorboard_callback = callbacks.TensorBoard(log_dir ="./logs")

In [1]:
autoencoder.fit(
    x_train,
    x_train,
    epochs = EPOCHS ,
    batch_size = BATCH_SIZE ,
    shuffle = True,
    VALIDATION_SPLIT

)

SyntaxError: ignored

In [2]:
#save the final models
autoencoder.save("./models/autoencoder")
encoder.save("./models/encoder")
decoder.save("./models/decoder")

NameError: ignored

#4. Reconstruction using the autoencoder

In [3]:
n_to_predict = 5000
example_images = x_test[:n_to_predict]
example_labels = y_test[:n_to_predict]



NameError: ignored

In [4]:
predictions = autoencoder.predict(example_images)

print("Example real clothing items")
display(example_images)
print("Reconstruction")
display(predictions)


NameError: ignored

#5. Embed using the encoder

In [5]:
#Encode the example images
embeddings = encoder.predict(example_images)


NameError: ignored

In [6]:
#some examples of the embeddings
print(embeddings[:10])

NameError: ignored

In [7]:
#show the encoded points in 2D space
figsize = 8

plt.figure(figsize = (figsize, figsize))
plt.scatter(embeddings[:, 0] , embeddings[:, 1] , c = "black" , alpha = 0.5 , s = 3)
plt.show()


NameError: ignored

In [8]:
#color the embeddings by their label(clothing type -)

example_labels = y_test[:n_to_predict]

figsize = 8
plt.figure(figsize = (figsize , figsize))
plt.scatter(
    embeddings[:, 0 ],
    embeddings[: , 1],
    cmap = "rainbow",
    c = example_labels,
    alpha = 0.8,
    s = 3,
)
plt.colorbar()
plt.show()

NameError: ignored

#6. Generate using the decoder

In [9]:
#Get the range of the existing embeddings
mins, maxs = np.min(embeddings, axis = 0) , np.max(embeddings, axis = 0)

#sample some points in the latent space
grid_width , grid_height = (6, 3)
sample = np.random.uniform(
    mins, maxs , size = (grid_width * grid_height, EMBEDDING_DIM)
)

NameError: ignored

In [10]:
#Decode the smapled points
reconstructions = decoder.predict(sample)

NameError: ignored

In [1]:
#Draw a plot of
figsize = 8
plt.figure(figsize = (figsize , figsize))

#... the orignal embeddings
plt.scatter(embeddings[:,0] , embeddings[:, 1] , c = "black" , alpha = 0.5 , s = 2)

#...and the only generated points in the latent space
plt.scatter(sample[:, 0] , sample[:, 1] , c = "#00B0F0" , alpha =1 , s = 40)
plt.show()

#add underneath a grid of the decoded images
fig = plt.figure(figsize= (figsize , grid_height * 2))
fig.subplots_adjust(hspace = 0.4 , wspace = 0.4)

for i in range(grid_width * grid_height):
  ax = fig.add_subplot(grid_height , grid_width , i +1)
  ax.axis("off")
  ax.text(
      0.5,
      -0.35,
      str(np.round(sample[i, :] , 1)),
      fontsize = 10,
      ha = "center",
      transform = ax.transAxes,
  )
  ax.imshow(reconstructions[i, : , :] , cmap = "Greys")


NameError: ignored

In [2]:
#color the embeddings by their label(clothing type )

figsize = 12
grid_size = 15
plt.figure(figsize = (figsize, figsize))
plt.scatter(
    embeddings[: , 0],
    embeddings[: , 1],
    cmap = "rainbow",
    c = example_labels,
    alpha = 0.8,
    s = 300,

)

plt.colorbar()

x = np.linespace(min(embeddings[:, 0]), max(embeddings[:, 0]) , grid_size)
y = np.linespace(max(embeddings[:, 1]), min(embeddings[:, 1]) , grid_size)
xv, yv = np.meshgrid(x,y)
xv = xv.flatten()
yv = yv.flatten()
grid = np.array(list(zip(xv, yv)))

reconstructions = decoder.predict(grid)
plt.show()


fig = plt.figure(figsize = (figsize, figsize))
fig.subplots_adjust(hspace = 0.4 , wspace = 0.4)
for i in range(grid_size ** 2):
  ax = fig.add_subplot(grid_size, grid_size, i + 1)
  ax.axis("off")
  ax.imshow(reconstructions[i , : , :], cmap = "Greys")

NameError: ignored